# Lab 5 - Introduction to Match-action Tables (Part 2)
<div style="text-align: justify;">
This lab will walk you through creating an experiment that contains a P4-DPDK programmable pipeline. This lab describes match-action tables and how to define them in a P4 program. It then explains the different types of matching that can be performed on keys. The lab further shows how to track the misses/hits of a table key while receiving a packet.
<figure style="text-align: center;">
<img src="./labs_files/lab5/figs/0_00_fabric_topology.png" width="700px"><br>

# Introduction

## Longest prefix match (LPM)

<div style="text-align: justify;">
Table 1 is an example of a match-action table that uses LPM [<a href="#References">1,2</a>]. Assume that the key is formed with the destination IP address. If an incoming packet has the destination IP address 172.168.3.5, two entries match. The first entry matches because the first 29 bits in the entry are the same as the first 29 bits of the destination IP. The second entry also matches because the first 16 bits in the entry are the same as the first 16 bits of the destination IP. The LPM algorithm will select 172.168.3.0/29 because of the longest prefix preference. </div>   <br>

<table style="width: 50%; font-size: 14px;">
  <caption>Table 1. Exact match table.</caption>
  <thead>
    <tr style="background-color: #f0f0f0;">
      <th>Key</th>
      <th>Action</th>
      <th>Action data</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="padding: 10px; text-align: center;">172.168.3.0/29</td>
      <td style="padding: 10px; text-align: center;">forward</td>
      <td style="padding: 10px; text-align: center;">port 1, macAddr=00:00:00:00:00:01</td>
    </tr>
    <tr>
      <td style="padding: 10px; text-align: center;">172.168.0.0/16</td>
      <td style="padding: 10px; text-align: center;">forward</td>
      <td style="padding: 10px; text-align: center;">port 2, macAddr=00:00:00:00:00:02</td>
    </tr>
    <tr>
      <td style="padding: 10px; text-align: center;">default</td>
      <td style="padding: 10px; text-align: center;">drop</td>
      <td style="padding: 10px; text-align: center;"> </td>
    </tr>
  </tbody>
</table>

Figure 1 shows the main control block portion of a P4 program [<a href="#References">3,4</a>]. Two actions are defined, ```drop``` and ```forward```. The ```drop``` action (lines 6 - 8) invokes the ```drop_packet``` function, causing the packet to be dropped. The ```forward``` action (lines 9 - 12) accepts as input (action data) the destination MAC address and port. These parameters are inserted by the control plane and updated in the packet during the ingress processing. In line 10, the P4 program assigns the egress port defined by the control plane as an input to the ```send_to_port``` extern function. It is used to direct a packet to a specified network port. Line 11 assigns the destination MAC address passed as a parameter to the packet's new destination address. Lines 13-23 implement a table named ```forwarding_lpm```. The table matches the destination IP address using the LPM type. The actions associated with the table are ```forward``` and ```drop```. The default action invoked when there is a miss is the drop action. The maximum number of entries is defined by the programmer (i.e., 1024 entries, see line 21). The control block starts executing from the apply statement (see lines 24-28) which contains the control logic. In this program, the ```forwarding_lpm``` table is activated in case the incoming packet has a valid IPv4 header.

<figure style="text-align: center;">
  <img src="./labs_files/lab5/figs/0_intro_1_p4_control.png" width="600" style="display: block; margin: 0 auto;">
  <figcaption>Figure 1. Main control block portion of a P4 program. The code implements a match-action table with LPM lookup.</figcaption>
</figure>

# Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

# Step 2: Importing the FABlib library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Creating the experiment slice

The following creates a node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Creating a slice
The code below creates a new slice with the name "P4DPDK_lab5_vnic"

In [2]:
slice = fablib.new_slice(name="P4DPDK_lab5_vnic")

### Step 3.2: Defining the sites
The code below requests a random site from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [3]:
site1= fablib.get_random_sites(count=1, filter_function=lambda x: x['cores_available'] > 4 and x['ram_available'] > 8 and x['disk_available'] > 20)[0]

print (f'The selected site is {site1}')

The selected site is KANS


### Step 3.3: Creating the nodes
The code below creates one node (server1) which uses the following:
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1

In [4]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [5]:
slice.submit();


Retry: 9, Time: 208 sec


ID,d5b66d5a-6e03-4a9a-8e0c-97f2847851a9
Name,P4DPDK_lab5_vnic
Lease Expiration (UTC),2024-09-05 14:05:33 +0000
Lease Start (UTC),2024-09-04 14:05:33 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
30466cc0-3783-4114-bde2-6d310b9e9ca3,server1,4,8,100,default_ubuntu_20,qcow2,kans-w1.fabric-testbed.net,KANS,ubuntu,2001:400:a100:3060:f816:3eff:fea7:dcf5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fea7:dcf5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


# Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the DPDK library, the P4 compiler (p4c), and all needed dependencies.

### Step 4.1 Getting the server node
The command below gets the fablib node that server1 is associated with.

In [6]:
server1 = slice.get_node(name="server1")

### Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [7]:
from ipaddress import ip_address, IPv6Address

if type(ip_address(server1.get_management_ip())) is IPv6Address:
    server1.upload_file('scripts/nat64.sh', 'nat64.sh')
    stdout, stderr = server1.execute(f'chmod +x nat64.sh && ./nat64.sh', quiet=True)

### Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [8]:
stdout, stderr = server1.execute(f'sudo apt-get update', quiet = True)
stdout, stderr = server1.execute(f'sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools', quiet = True)
stdout, stderr = server1.execute(f'sudo pip3 install meson ninja scapy', quiet = True)

### Step 4.4 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers. In this lab, we are building a modified version of DPDK in which we enabled logs in the terminal to have a better understanding of the behavior of the built pipeline. Note that printing out logs on the terminal degrades performance. Therefore, with applications where high performance is needed, logging should be disabled.  

In [9]:
stdout, stderr = server1.execute(f'git clone http://dpdk.org/git/dpdk', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
server1.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
server1.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
stdout, stderr = server1.execute(f'cd dpdk &&  sudo meson build && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)
stdout, stderr = server1.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

### Step 4.5 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline

In [10]:
stdout, stderr = server1.execute('source /etc/lsb-release && echo "deb http://download.opensuse.org/repositories/home:/p4lang/xUbuntu_${DISTRIB_RELEASE}/ /" | sudo tee /etc/apt/sources.list.d/home:p4lang.list && curl -fsSL https://download.opensuse.org/repositories/home:p4lang/xUbuntu_${DISTRIB_RELEASE}/Release.key | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/home_p4lang.gpg > /dev/null && sudo apt-get update && sudo apt install -y p4lang-p4c', quiet = True)
stdout, stderr = server1.execute("cd /usr/share/p4c/p4include/dpdk/ && sudo sed -i '769s/\\(.\\{6\\}\\)/\\1out/' pna.p4", quiet=True)

# Step 5: Writing and compiling the P4 code
This section demonstrates how to implement a simple table in P4 that uses exact matching on the destination IP address of the packet. When there is a match, the packet is forwarded from a certain port. If no match is found, LPM is applied to the packet’s destination IP address. If a match is found, the packet is forwarded from a certain port. Otherwise, the packet is dropped.

## Step 5.1: Programming the control block

Click on [control.p4](./labs_files/lab5/control.p4) to open the code in the editor.

<img src="./labs_files/lab5/figs/5_01_01.png" width="700px"><br>

We can see that the control.p4 declares a control block named MainControl. Note that the body of the control block is empty. Our objective is to define P4 tables and their actions, and then invoke them inside the block.

<hr>

Now we will define the behavior of the ```forward_exact``` action. Insert the code below inside the ```MainControl``` control block. 

    action forward_exact (EthernetAddress dstAddr, PortId_t port_id) {
        send_to_port(port_id);
        hdr.ethernet.srcAddr = hdr.ethernet.dstAddr;
        hdr.ethernet.dstAddr = dstAddr;
        hdr.ipv4.ttl = hdr.ipv4.ttl - 1;
    }

<img src="./labs_files/lab5/figs/5_01_02.png" width="700"><br>

The action ```forward_exact``` accepts as parameters the next hop’s MAC address (i.e., ```EthernetAddress dstAddr```) and the port number (```PortId_t port_id```) to be used by the pipeline to forward the packet. Note that ```EthernetAddress``` is a typedef that corresponds to ```bit<48>``` defined in the ```headers.p4``` file. 

The ```send_to_port``` is a function that takes the port number as an input. Therefore, when the forward action is executed, the packet will be sent out of the port number specified as a parameter.

To modify header fields inside the packet, we refer to the field name based on where it exists inside the headers. Recall that the names of the headers and the fields are defined by the programmer. The file headers.p4 defines the program’s headers. Line 10 shows how we are assigning the destination MAC address of the packet (i.e., ```hdr.ethernet.dstAddr```) to be the new source MAC of the packet (i.e., ```hdr.ethernet.srcAddr```). Line 11 shows how we are assigning the destination MAC address which is provided as a parameter (assigned later in the control plane) to be the new destination MAC of the packet. 

It is possible in P4 to perform basic arithmetic operations on header fields and other variables. In line 12, we are decrementing the TTL value of the header field.

<hr>

Similarly, we will define the behavior of the forward_lpm action. Insert the code below inside the MainControl control block. 

    action forward_lpm (EthernetAddress dstAddr, PortId_t port) {
        send_to_port(port_id);
        hdr.ethernet.srcAddr = hdr.ethernet.dstAddr;
        hdr.ethernet.dstAddr = dstAddr;
        hdr.ipv4.ttl = hdr.ipv4.ttl - 1;
    }

<img src="./labs_files/lab5/figs/5_01_03.png" width="700"><br>

Note that the P4-DPDK compiler matches each action to one table. In this application, we will need two tables, one for exact matching and the other for LPM. Therefore, a forwarding action is needed for each (forward_exact and forward_lpm).

<hr>

Now we will define the drop action. Insert the code below inside the MainControl control block. 

    action drop() {
        drop_packet();
    }

<img src="./labs_files/lab5/figs/5_01_04.png" width="700"><br>

The drop() action invokes a primitive action drop_packet() that causes the packet to be dropped.

<hr>

Now we will define the table named forwarding_exact. Write the following piece of code inside the body of the MainControl control block.
    
    table forwarding_exact {
        key = {
            hdr.ipv4.dstAddr: exact;
        }
        actions = {
            forward_exact;
        }
        size = 1024;
    }


<img src="./labs_files/lab5/figs/5_01_05.png" width="700"><br>

The inserted code specifies that the destination IPv4 address of a packet (```hdr.ipv4.dstAddr```) will be used as a key in the ```forwarding_exact``` table. Note that the match type is ```exact```, denoting that the value of the destination IP address will be matched as is against a value specified later in the control plane. The defined possible action is the ```forward_exact``` action. Finally, the ```size``` keyword specifies the maximum number of entries that can be inserted into this table from the control plane.

<hr>

Now we will define a table that performs an LPM on the destination IP address of the packet. The table will be invoking the forward and the drop actions, and hence, those actions will be listed inside the table definition.

    table forwarding_lpm {
        key = {
            hdr.ipv4.dstAddr: lpm;
        }
        actions = {
            forward_lpm;
            drop;
        }
        size = 1024;
        default_action = drop();
    }

<img src="./labs_files/lab5/figs/5_01_06.png" width="700"><br>

The inserted code specifies that the destination IPv4 address of a packet (```hdr.ipv4.dstAddr```) will be used as a key in the ```forwarding_lpm``` table. Note that the match type is ```lpm```, denoting that the value of the destination IP address will be matched based on the LPM algorithm. The defined possible actions are the ```forward_lpm``` and the ```drop``` actions. The ```size``` keyword specifies the maximum number of entries that can be inserted into this table from the control plane. Finally, the default action that is invoked when there is a miss is ```drop``` action.

<hr>

Add the following code at the end of the main control block. The apply block defines the sequential flow of packet processing. It is required in every control block, otherwise the program will not compile. It describes the sequence of tables to be invoked, in addition to other packet processing instructions.

    apply {
        if(hdr.ipv4.isValid()) {
            if(forwarding_exact.apply().miss) {
                forwarding_lpm.apply();
            }
        }
    }

<img src="./labs_files/lab5/figs/5_01_07.png" width="700"><br>

The logic of the code above is as follows: if the packet has an IPv4 header, apply the ```forwarding_exact``` table which performs an exact match lookup on the destination IP address. If there is no hit (i.e., the table does not contain a rule that corresponds to this IPv4 address, denoted by the miss keyword), apply the ```forwarding_lpm``` table, which matches the destination IP address of the packet against a network address. At this stage, if no match is found by the LPM algorithm, the packet is dropped when the default action in the LPM table is invoked.

<hr>

Save the changes by pressing ```Ctrl+s```.

## Step 5.2: Compiling the P4 code
To upload all needed P4 codes and compile main.p4, issue the following command.

In [12]:
server1.upload_file('labs_files/lab5/headers.p4','headers.p4')
server1.upload_file('labs_files/lab5/parser.p4','parser.p4')
server1.upload_file('labs_files/lab5/precontrol.p4','precontrol.p4')
server1.upload_file('labs_files/lab5/control.p4','control.p4')
server1.upload_file('labs_files/lab5/deparser.p4','deparser.p4')
server1.upload_file('labs_files/lab5/main.p4','main.p4')
stdout, stderr = server1.execute(f'sudo p4c-dpdk --arch=pna main.p4 -o lab5.spec')
stdout, stderr = server1.execute(f'ls')

 sudo: unable to resolve host server1: Name or service not known
 control.p4
deparser.p4
dpdk
headers.p4
host_tune.sh
lab5.spec
main.p4
nat64.sh
p4c
parser.p4
precontrol.p4


The command above invokes the ```p4c-dpdk``` compiler to compile the ```main.p4``` program and generates the ```lab5.spec``` file which is a specification file needed to run the pipeline.

# Step 6: Running the P4-DPDK pipeline and the lab topology
This section shows the steps required to run the P4-DPDK along with building the lab topology. In this lab, the procedure is automated.

## Step 6.1: Uploading required scripts
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

We will also upload the rules.txt file to load and add rules to the forwarding table. The rules.txt file contains rules for exact matching.

In [13]:
server1.upload_file('labs_files/lab5/lab5.cli','lab5.cli')
server1.upload_file('labs_files/lab5/ethdev.io','ethdev.io')
server1.upload_file('labs_files/lab5/run_pipeline.sh','run_pipeline.sh')
server1.upload_file('labs_files/lab5/set_topology.sh','set_topology.sh')
server1.upload_file('labs_files/lab5/sender.py','sender.py')
server1.upload_file('labs_files/lab5/rules_exact.txt','rules_exact.txt')
server1.upload_file('labs_files/lab5/rules_lpm.txt','rules_lpm.txt')
stdout, stderr = server1.execute(f'chmod +x run_pipeline.sh')
stdout, stderr = server1.execute(f'chmod +x set_topology.sh')
stdout, stderr = server1.execute(f'chmod +x sender.py')

## Step 6.2: Opening a terminal
Launch a new terminal by opening a new tab and then select "terminal".

<img src="./labs_files/lab5/figs/6_02_terminal.gif" width="1000px"><br>

Copy the output of the command below and paste it into the terminal to enter server1.

In [14]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3060:f816:3eff:fea7:dcf5'

## Step 6.3: Running the P4-DPDK pipeline
Build and run the pipeline by typing the following command:

    sudo ./run_pipeline.sh
    
<img src="./labs_files/lab5/figs/6_03.png" width="750px"><br>

The ```run_pipeline.sh``` script is a shell script that automates the process of running the P4-DPDK pipeline. In this lab, part of the pipeline output is hidden to display only the relevant logs.

## Step 6.4: Building the lab topology
The code below uploads to server1 the CLI and I/O scripts, along with other scripts needed to automate the process of running the pipeline and building the topology.

In [15]:
stdout, stderr = server1.execute(f'sudo ./set_topology.sh')

 sudo: unable to resolve host server1: Name or service not known
 

The set_topology.sh script is a shell script that automates the process of building the lab topology. Two namespaces are built and configured in this step with a virtual device linked to each as shown in the figure below.

<img src="./labs_files/lab5/figs/6_04_ip.png" width="650px"><br>

## Step 6.5: Inspect the rules_exact.txt file
Click on [rules_exact.txt](./labs_files/lab5/rules_exact.txt) to open the code in the editor.

<img src="./labs_files/lab5/figs/6_05.png" width="550px"><br>

The figure above shows the entry loaded to the forwarding_exact match action table. The entry consists of three components.

•	```match <key>```: The key based on which a match is found or not. The key is 192.168.30.1 (0xC0A81e01 in hexadecimal) which is the IP address of host 3. <br>
•	```action <action name>```: The action to be executed if there is a match. If a match is found the action to be executed is the forward_exact action declared in the control.p4 code which was written earlier in this lab. <br>
•	The last item is the action data. In this case, the action data are the destination MAC address and egress port ID. Therefore, if a match is found with this entry, the packet is forwarded to host 3 through port 2 (0x2 in hexadecimal), and the destination MAC address is 00:00:00:00:00:03 (0x3 in hexadecimal).


## Step 6.6: Inspect the rules_lpm.txt file
Click on [rules_lpm.txt](./labs_files/lab5/rules_lpm.txt) to open the code in the editor.

<img src="./labs_files/lab5/figs/6_06.png" width="550px"><br>

The figure above shows the entries loaded to the forwarding_lpm match action table. Each entry consists of four components.

•	```match <key>```: The key based on which a match is found or not. In the first entry, the key is 192.168.10.0/24 (0xC0A80A00/ffffff00 in hexadecimal) which is the IP address and netmask of host 1. The match key of the second entry is 192.168.20.0/24 (0xC0A81400/ffffff00 in hexadecimal) which is the IP address and netmask of host 2. <br>
•	```action <action name>```: The action to be executed if there is a match. If a match is found the action to be executed for both entries is the forward_lpm action declared in the control.p4 code which was written earlier in this lab. <br>
•	The last item is the action data. In this case, the action data are the destination MAC address and egress port ID. Therefore, if a match is found with the first entry, the packet is forwarded to host 1 through port 0 (0x0 in hexadecimal) and the destination MAC address is 00:00:00:00:00:01 (0x1 in hexadecimal). Whereas, if a match is found with the second entry, the packet is forwarded to host 2 through port 1 (0x1 in hexadecimal) and the destination MAC address is 00:00:00:00:00:02 (0x2 in hexadecimal).

# Step 7: Testing exact matching

### Step 7.1: Testing exact matching
To test exact matching, we will send a packet with an ethernet, IPv4, and TCP header from h1 to h3 over the destination IP address 192.168.30.1 by running the provided Python script sender.py.

In [16]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.30.1')

. sudo: unable to resolve host server1: Name or service not known
 Sender: h1
Destination IP: 192.168.30.1

Sent 1 packets.


Running the Python script requires two parameters:

•	```-s```: Sender (h1 or h2 or h3) <br>
•	```-d```: Destination IP Address



### Step 7.2: Inspect the pipeline terminal

Observe the DPDK logs at the bottom of the terminal. These logs correspond to the packet processing function executed in the .spec file generated when the P4 code is compiled. 

<img src="./labs_files/lab5/figs/7_02.png" width="700px"><br>

The log highlighted in the first grey box corresponds to the match found in the match-action table with table ID 0 which indicates that an exact match is found. Since there is a match the forward_exact action with action ID 1 is executed. 

The second grey box shows that two move functions and one arithmetic operation are done to flip the source and destination MAC addresses and decrease the TTL as specified in the forward_exact action in the control.p4 script.

The “tx 1 pkt to port 2” log in the third grey box indicates that the forwarding action is properly executed by sending one packet to port 2 which corresponds to host 3.

For a more readable output press enter in the terminal a few times (five times) to provide space for the next logs.

### Step 7.3: Testing default action when no match is found
Now, we will send a packet with an ethernet, IPv4, and TCP header from h1 to h3 over the destination IP address 192.168.30.3 by running the provided Python script sender.py.

In [17]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.30.3')

. sudo: unable to resolve host server1: Name or service not known
 Sender: h1
Destination IP: 192.168.30.3

Sent 1 packets.


### Step 7.4: Inspect the pipeline terminal

<img src="./labs_files/lab5/figs/7_04.png" width="700px"><br>

The log highlighted in the first grey box corresponds to the table lookup. An exact match was not found in table 0 which corresponds to the forwarding_exact table. In this case, a table lookup will be performed in the second table named forwarding_lmp with a table ID of 1. A match was not found in the second table. Therefore, the default action (action 3) is executed, and the packet is dropped. The log in the second grey box indicates that one packet has been dropped.

For a more readable output press enter in the terminal a few times (five times) to provide space for the next logs.

### Step 7.5: Testing LPM
To test the LPM table lookup, we will send a packet with an ethernet, IPv4, and TCP header from h1 to h2 over the destination IP address 192.168.20.3 by running the provided Python script sender.py.

In [18]:
stdout, stderr = server1.execute(f'sudo ip netns exec h1 python3 sender.py -s h1 -d 192.168.20.3')

. sudo: unable to resolve host server1: Name or service not known
 Sender: h1
Destination IP: 192.168.20.3

Sent 1 packets.


### Step 7.6: Inspect the pipeline terminal

<img src="./labs_files/lab5/figs/7_06.png" width="700px"><br>

The log highlighted in the first grey box corresponds to the table lookup. An exact match was not found in table 0 which corresponds to the forwarding_exact table. In this case, a table lookup will be performed in the second table named forwarding_lmp with a table ID of 1. A match was found in the second table. Therefore, the forward_lpm (action 2) is executed.

The second grey box shows that two move functions and one arithmetic operation are done to flip the source and destination MAC addresses and decrease the TTL as specified in the forward_lpm action in the control.p4 script.

The “tx 1 pkt to port 1” log in the third grey box indicates that the forwarding action is properly executed by sending one packet to port 1 which corresponds to host 2.

### Step 7.7: Stopping the DPDK pipeline
Stop the DPDK pipeline by pressing ```ctrl+c``` in the terminal running the pipeline.

<img src="./labs_files/lab5/figs/7_07.png" width="700px"><br>

# Step 8: Delete the slice

This concludes Lab 5. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_lab5_vnic")
slice.delete()

# References

1.	The P4 Language Consortium, “P4 Portable NIC Architecture (PNA)”, Version 0.5, 2021. [Online]. Available: https://p4.org/p4-spec/docs/PNA.html
2.	The P4 Language Consortium, “P4 Portable Switch Architecture (PSA)”, 2021. [Online]. Available: https://p4.org/p4-spec/docs/PSA.html#sec-match-kinds
3.	P4lang, “pna”, [Online]. Available: https://github.com/p4lang/pna/tree/main?tab=readme-ov-file
4.	“p4c core.p4”. [Online]. Available: https://github.com/p4lang/p4c/blob/main/p4include/core.p4.